In [2]:
import pandas as pd

df_ann = pd.read_csv('./annotated_pagos/areiospagos_annotated.csv')

In [3]:
df_ann

,Decision,Label
0,Decision_0,1
1,Decision_1,0
2,Decision_2,1
3,Decision_3,1
4,Decision_4,1
...,...,...
2995,Decision_2995,1
2996,Decision_2996,1
2997,Decision_2997,1
2998,Decision_2998,1


In [3]:
df_ann['Label'].value_counts()

Label
1    2450
0     550
Name: count, dtype: int64

Creating a new balanced dataset to work with for Areios Pagos


In [4]:
df_0 = df_ann[df_ann['Label'] == 0]
df_1 = df_ann[df_ann['Label'] == 1]

df_0_sample = df_0.sample(500)
df_1_sample = df_1.sample(550)

df_resampled_ap = pd.concat([df_0_sample, df_1_sample])

# Shuffle the dataset
df_resampled_ap = df_resampled_ap.sample(frac=1).reset_index(drop=True)

# Check the distribution of labels
df_resampled_ap['Label'].value_counts()


Label
1    550
0    500
Name: count, dtype: int64

In [5]:
df_resampled_ap

,Decision,Label
0,Decision_2825,0
1,Decision_2739,0
2,Decision_2275,1
3,Decision_1285,1
4,Decision_12,0
...,...,...
1045,Decision_776,1
1046,Decision_1612,1
1047,Decision_2298,1
1048,Decision_392,1


In [12]:
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec
import logging  # Setting up the loggings to monitor gensim
import re
import unicodedata as ud
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
import unicodedata

def preprocess_text(file_path):
    with open(file_path, 'r') as file:
        # Read the text from the file
        text = file.read()
        # Convert to lowercase
        text = text.lower()
        # Remove accents
        text = ''.join(char for char in unicodedata.normalize('NFKD', text) if not unicodedata.combining(char))
        # Remove punctuations
        text = re.sub(r'[^\w\s]', '', text)
        # Remove numbers
        text = re.sub(r'\d+', '', text)
        # Remove English characters
        text = re.sub(r'[a-zA-Z]', '', text)
        # Remove all special chars
        text = text.replace('_', '')
        # Transforn to list of words
        text = text.replace('\n', ' ').split(' ')
    return text
stop_words = preprocess_text('stopwords.txt')



## TF-IDF VECTORIZER ##
TF-IDF Vectorization:

It uses TfidfVectorizer from scikit-learn to convert the text data (df_resampled['script']) into a matrix of TF-IDF features.
The parameters used in TfidfVectorizer are:

norm='l2': This normalizes the vectors to have an L2 norm (unit norm).

stop_words=stop_words: This removes common stop words (as defined by the stop_words variable) from the text before vectorizing.

max_features=5000: Limits the vocabulary to the 5000 most frequent terms.

Transformation:

The fit_transform method is applied to the text data (df_resampled['script']), which learns the vocabulary and IDF (Inverse Document Frequency) values from the data, and then transforms the text data into a TF-IDF weighted term-document matrix.
This matrix is then converted into a dense array using .toarray().

Creating a DataFrame:

The TF-IDF matrix is then converted into a Pandas DataFrame (data_resampled), with the same index as the original DataFrame (df_resampled.index) and with columns named after the terms in the vocabulary (tfidf.get_feature_names_out()).
Appending the Label:

The 'Label' column from the original df_resampled DataFrame is added to the data_resampled DataFrame, aligning by index.





In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
import unicodedata

stop_words = preprocess_text('stopwords.txt')

tfidf = TfidfVectorizer(norm='l2', stop_words=stop_words, max_features=5000)
tfidf_matrix = tfidf.fit_transform(df_resampled_ap['script']).toarray()

data_resampled_ap = pd.DataFrame(tfidf_matrix, index=df_resampled_ap.index, columns=tfidf.get_feature_names_out())
data_resampled_ap['Label'] = df_resampled_ap['Label'].values


KeyError: 'script'